In [43]:
import torch
import pickle
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 64
batch_size = 128
max_iters = 3000
learning_rate = 3e-4 # 3e-3, 1e-3, 1e-4
eval_iters = 100
eval_interval = 500
n_embd = 384
n_layer = 4
n_head = 4
dropout = 0.2


cpu


In [44]:
chars = ""
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()
    chars = sorted(set(text))
    
vocab_size = len(chars)

In [45]:
string_to_int = { ch:i for i, ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
#print(data[:100])

In [46]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
#print('Inputs:')
#print(x)
#print('Targets:')
#print(y)



In [47]:
@torch.no_grad()
def estimate_loss():
    '''estimate loss'''
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out
        

In [48]:
class Head(nn.Module):
    '''head'''

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias = False)
        self.query = nn.Linear(n_embd, head_size, bias = False)
        self.value = nn.Linear(n_embd, head_size, bias = False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x)
        q = self.query(x)

        wei = q @ k.transpose(-2, -1) * k.shape[-1]**-0.5
        wei = wei.masked_fill(self.tril[:T, :T] ==0, float('-inf'))
        wei = F.softmax(wei, dim = -1)
        wei = self.dropout(wei)

        v = self.value(x)
        out = wei @ v
        return out
    


In [49]:
class MultiHeadAttention(nn.Module):
    '''Multi Head'''

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim = -1)
        out = self.dropout(self.proj(out))
        return out
    

In [50]:
class FeedForward(nn.Module):
    '''Feed Forward'''
    
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

In [51]:
class Block(nn.Module):
    '''Transformer Block'''
    
    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_head // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        y = self.sa(x)
        x = self.ln1(x + y)
        y = self.ffwd(x)
        x = self.ln2(x + y)
        return x

In [54]:
class GPTLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)

        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)
        
        self.apply(self._init_weights)

    def _init_weights(self, module):
        '''init weights'''
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
            elif isinstance(module, nn.Embedding):
                torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, index, targets=None):
        '''forward'''
        B, T = index.shape

        tok_emb = self.token_embedding_table(index)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device))
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self, index, max_new_tokens):
        '''generate tokens'''
        for _ in range(max_new_tokens):
            index_cond = index[:, -block_size:]
            logits, loss = self.forward(index_cond)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            index_next = torch.multinomial(probs, num_samples=1)
            index = torch.cat((index, index_next), dim=1)
        return index

model = GPTLanguageModel(vocab_size)
m = model.to(device)    


In [55]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"Step: {iter}, Train Loss: {losses['train']:.3f}, Val Loss: {losses['val']:.3f}")

    xb, yb = get_batch('train')

    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none = True)
    loss.backward()
    optimizer.step()

print(loss.item())

with open('model-01.pkl', 'wb') as f:
    pickle.dump(model, f)
print('model saved')



Step: 0, Train Loss: 4.431, Val Loss: 4.431
Step: 100, Train Loss: 2.453, Val Loss: 2.510
Step: 200, Train Loss: 2.350, Val Loss: 2.418
Step: 300, Train Loss: 2.205, Val Loss: 2.285
Step: 400, Train Loss: 2.127, Val Loss: 2.218
Step: 500, Train Loss: 2.045, Val Loss: 2.146
Step: 600, Train Loss: 1.983, Val Loss: 2.091
Step: 700, Train Loss: 1.915, Val Loss: 2.028
Step: 800, Train Loss: 1.869, Val Loss: 1.990
Step: 900, Train Loss: 1.826, Val Loss: 1.951
Step: 1000, Train Loss: 1.786, Val Loss: 1.917


In [ ]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
#print(generated_chars)



OTw6Gh&[;5TKvwS&Aj9iA:w6isMnU﻿YkXc"Md"'d,FZ;0'U1iH[uEj:Ld[)k7W(QYedi-_n;P5(8u,NxCDz)o]dz?rOV:h9fB;yf
StMr[s8sP5[eloTZ5AL
S vdWXS
7oz5FVpUqkNfp:ROfyorlx3U-2M'S)qVk)GXtS0tKC0:3IsE6(rzYOFe7E]WXt'XS8u(88GlMF?_3,au"L6k)oDoo

Y,i936KbF0e KXm﻿dkQ4)pi)AK1[fU:D"BMrek"M"wG(3D)﻿e91[.rP1BACDi:Dz4ykkA
7nQJZJ3lCx1D7(P1RX0&BykAVVy 'atKK﻿zCA8uvY!KR]I_GOSRsNmdTT]P;pp(3!KR﻿m,9.JD78i1B0
Oxcw-KW'65d9s4M5B-JL63)ai1!Tf'2J.V30EM[TPQRwr5Mr93pEV98ufL:5(!dytp0FZMh:wauQw!1w2MFg(i'zSLY5zY6t-1nHg8)bcT nXxNQCRT2lD'6t4&,U?(cy
